In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_csv('Original Data/brand_survey_feb.csv')

raw_df.head()

,Date & Time,Duration,NPS,"Other than Twinkl, which ONE teaching resource brand that you use comes to mind?",What other teaching resource brands that you use come to mind?,"Including any of the teaching resource brands you mentioned, which of the following brands have you heard of?",How likely or unlikely are you to consider Twinkl the next time you need resources? -,"If your favorite brand isn’t available/doesn’t offer what you need, which other brands would you consider?",What three words best describe the Twinkl brand?,Word 1,...,Career,County,Subscription Type,User,Bundle,Career.1,Career Category,Simplified Career,Country.1,County.1
0,"February 9, 2025 22:46:46",0:5:42 Hrs,8,nearpod,Teacher-pay-teacher,"Teachers Pay Teachers,Khan Academy,McGraw-Hill",Likely,Khan Academy,NaN,Varity,...,787,1797,3.0,22000858,Ultimate,Grade 6 Teacher,US Grade 6,Teacher,United States,Washington
1,"February 10, 2025 02:04:15",0:7:40 Hrs,10,Teacher pay Teacher,Teacher pay Teacher,"Teachers Pay Teachers,Scholastic,Khan Academy,...",Very Likely,"Twinkl, now that I found it.",NaN,Relavent,...,784,177,3.0,25361861,Ultimate,Grade 3 Teacher,US Grade 3,Teacher,United States,Indiana
2,"February 10, 2025 05:02:15",0:45:48 Hrs,8,IMSE OG,\n,"Teachers Pay Teachers,Scholastic,Khan Academy,...",Very Unlikely,ooops...hit the wrong button on the last one.....,NaN,engaging,...,516,192,3.0,10210824,Ultimate,Special Education Teacher,sen,Teacher,United States,New Hampshire
3,"February 10, 2025 05:21:15",0:9:52 Hrs,10,TPT,TPT,"Teachers Pay Teachers,Scholastic,McGraw-Hill,P...",Neutral,I just google search for it,NaN,User friendly,...,31,193,NaN,22184678,Old Bundle,Kindergarten,US Kindergarten,Teacher,United States,New Jersey
4,"February 10, 2025 02:54:22",0:12:28 Hrs,9,NaN,NaN,"Teachers Pay Teachers,Scholastic,Khan Academy,...",Likely,Teachers pay teachers,NaN,Wonderful,...,30,168,3.0,25206512,Ultimate,Pre-Kindergarten,US Pre K,Teacher,United States,California


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 47 columns):
 #   Column                                                                                                                                                    Non-Null Count  Dtype  
---  ------                                                                                                                                                    --------------  -----  
 0   Date & Time                                                                                                                                               563 non-null    object 
 1   Duration                                                                                                                                                  563 non-null    object 
 2   NPS                                                                                                                                                       563 

In [4]:
questions = raw_df.columns

"""
Print with:

for idx, q in enumerate(questions):
    print(f"{idx}: {q}")
"""

'\nPrint with:\n\nfor idx, q in enumerate(questions):\n    print(f"{idx}: {q}")\n'

# Competitors List

In this section we'll extract a list of the top 10 - 20 most mentioned competitors from the survey responses.

There can be found on column indices `3` and `4`

In [5]:
competitor_mentions = raw_df.iloc[:,[3,4]].copy().reset_index(drop=True)

competitor_mentions.head()

,"Other than Twinkl, which ONE teaching resource brand that you use comes to mind?",What other teaching resource brands that you use come to mind?
0,nearpod,Teacher-pay-teacher
1,Teacher pay Teacher,Teacher pay Teacher
2,IMSE OG,\n
3,TPT,TPT
4,NaN,NaN


In [6]:
competitor_mentions.columns = ['primary', 'secondary'] + list(competitor_mentions.columns[2:])

competitor_mentions.columns

Index(['primary', 'secondary'], dtype='object')

In [7]:
print(competitor_mentions.isna().sum())

print(len(competitor_mentions))

primary       45
secondary    162
dtype: int64
563


In [8]:
competitor_mentions.dropna(subset='primary', inplace=True)

len(competitor_mentions)

518

In [9]:
import string

def preprocess_text(text):
    if not isinstance(text, str):
        return text
        
    text = text.lower()
    text = text.translate(str.maketrans('','', string.punctuation))

    return ' '.join(text.split())

In [10]:
competitor_mentions = competitor_mentions.map(preprocess_text)

competitor_mentions.head()

,primary,secondary
0,nearpod,teacherpayteacher
1,teacher pay teacher,teacher pay teacher
2,imse og,
3,tpt,tpt
6,educationcom,NaN


In [11]:
raw_competitor_mentions = competitor_mentions['primary'].value_counts().index.sort_values()

In [49]:
canonical_names = [
    '3Dinosaurs',
    'ABCMouse',
    'AdaptedMind',
    'All-in-One Homeschool',
    'Blooket',
    'Blossom and Root',
    'BookCreator',
    'Boom Cards',
    'Bright Futures Counseling',
    'Brisk Teaching',
    'Canva',
    'Carly and Adam',
    'Carson Dellosa Education',
    'ChatGPT',
    'Christian Ministry Deals',
    'Classroomscreen',
    'Code.org',
    'CodeHS',
    'Common Core Sheets',
    'CommonLit',
    'Crafts On Sea',
    'Education to the Core',
    'Education.com',
    'Ellii',
    'English Worksheets Land',
    'Epic',
    'Equipping ELLs',
    'Ereading Worksheets',
    'Etsy',
    'Evan Moor',
    'Extra Practice',
    'Facebook',
    'Flocabulary',
    'Functional Education',
    'Generation Genius',
    'Gimkit',
    'Ginger Tiger',
    'Google',
    'Harbor and Sprout',
    'Hooked on Phonics',
    "I'm Lovin Lit",
    'IMSE',
    'IXL',
    'Imagine Learning',
    'Itty Bitty Speech',
    'K12',
    'K5 Learning',
    'Kahoot',
    'Khan Academy',
    'Kindergarten Connection',
    'Kuta',
    'Lakeshore',
    'Learn Bright',
    'Learning A-Z',
    'Lexia',
    'Linguahouse',
    'Little Bins for Little Hands',
    'Lucky Little Learners',
    'Magic School',
    'Math Worksheets 4 Kids',
    'Math-Aids.Com',
    'Math-Drills',
    'Mathseeds',
    'Mondo Lama',
    'Nearpod',
    'Origami Organelles',
    'Padlet',
    'Pearson',
    'Pflaum Gospel Weeklies',
    'Phonics.com',
    'Pinterest',
    'Play to Learn Preschool',
    'Pocket of Preschool',
    'Printables.com',
    'Quizizz',
    'Read Theory',
    'ReadWorks',
    'SLP Now',
    'STAR Autism',
    'Scholastic',
    'SchoolAI',
    'Scribd',
    'Simple Everyday Mom',
    'Social Studies Success',
    'SparkleBox',
    'Study.com',
    'Super Teacher Worksheets',
    'Teach This',
    'Teachers Pay Teachers',
    'Teaching Tools',
    'Teaching with a Mountain View',
    'TeachingBooks',
    'The Autism Helper',
    'The Crafty Classroom',
    'The Good And The Beautiful',
    'The Mailbox',
    'The Measured Mom',
    'The OT Toolbox',
    'The Teachers Toolbox',
    'Timberdoodle',
    'TinyTap',
    'Ultimate SLP',
    'Whiterose',
    'WordMint',
    'Wordwall',
    'Your Therapy Source',
    'Youtube',
    'Zearn',
    'edHelper',
    'iReady',
    'iSL Collective',
    'n2y',
]


var_map = {
        'tpt' : 'teachers pay teachers',
        'tpt' : 'teachers pay teachers',
        'news to you' : 'n2y',
        'esl library' : 'ellii',
        'esllibrary' : 'ellii',
        'epiccom' : 'epic',
        'getepiccom' : 'epic',
        'imlovinlit' : "i'm lovin lit",
        'educationcom' : 'education.com',
        }


    


In [53]:
from rapidfuzz import process, fuzz
import re

normalized_canonical = {name.lower(): name for name in canonical_names}


def normalize_and_map(name, mapping, canonical_list, threshold=70):
    """
    Normalize the name and try to map it using an explicit dictionary.
    If no mapping exists, use fuzzy matching against the canonical list.
    """
    norm_name = name.lower().strip()
    if norm_name in mapping:
        return mapping[norm_name]
    
    # Use fuzzy matching to compare against canonical names (case-insensitive)
    match, score, _ = process.extractOne(norm_name, canonical_list, scorer=fuzz.token_set_ratio)
    if score >= threshold:
        return match
    else:
        return name  # Return the original input if no match is good enough

def process_company_names(entry, mapping, canonical_list, delimiters=r'\band\b|,|/'):
    """
    Split the input string on common delimiters, process each fragment with normalization,
    mapping, and fuzzy matching, and reassemble the results.
    """
    # Split the input on delimiters; note: we lower-case early to standardize splitting.
    parts = re.split(delimiters, entry.lower())
    parts = [part.strip() for part in parts if part.strip()]
    
    # Process each fragment
    canonical_parts = [normalize_and_map(part, mapping, canonical_list) for part in parts]
    
    # Remove duplicates while preserving order
    seen = set()
    unique_canonicals = []
    for name in canonical_parts:
        if name not in seen:
            unique_canonicals.append(name)
            seen.add(name)
    
    return ", ".join(unique_canonicals)


competitor_mentions['canonical_name'] = competitor_mentions['primary'].apply(lambda x: process_company_names(x, var_map, canonical_names))

company_counts = competitor_mentions['canonical_name'].value_counts()

In [54]:
pd.set_option('display.max_rows', None)
competitor_mentions[['primary', 'canonical_name']]

,primary,canonical_name
0,nearpod,Nearpod
1,teacher pay teacher,Teachers Pay Teachers
2,imse og,imse og
3,tpt,teachers pay teachers
6,educationcom,education.com
7,pencil control activities,pencil control activities
8,schoolai,schoolai
9,phonics,phonics
10,mailbox,mailbox
11,equipping ells,Equipping ELLs


In [36]:
competitor_mentions[competitor_mentions['primary'].str.contains('library', case=False, na=False)]

,primary,secondary,canonical_name
425,esllibrary,english central,Ellii
